<a href="https://colab.research.google.com/github/DinDev3/Python-Machine-Learning/blob/master/Machine-Learning-with-Python/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

Install packages
- pip install sklearn
- pip install pandas
- pip install quandl

---
Stock Prices


In [0]:
# !pip --version
# !pip install --upgrade pip
!pip install sklearn
!pip install pandas
!pip install quandl

In [0]:
import pandas as pd
import quandl
import math

In [20]:
# storing the tabled data received from quandl in a pandas dataframe
df = quandl.get("WIKI/GOOGL")       # https://www.quandl.com/

print(df.head())

              Open    High     Low  ...   Adj. Low  Adj. Close  Adj. Volume
Date                                ...                                    
2004-08-19  100.01  104.06   95.96  ...  48.128568   50.322842   44659000.0
2004-08-20  101.01  109.08  100.50  ...  50.405597   54.322689   22834300.0
2004-08-23  110.76  113.48  109.05  ...  54.693835   54.869377   18256100.0
2004-08-24  111.24  111.60  103.57  ...  51.945350   52.597363   15247300.0
2004-08-25  104.76  108.00  103.88  ...  52.100830   53.164113    9188600.0

[5 rows x 12 columns]


Each column is a feature here.

All these columns aren't needed to find a pattern.

Relationships between features can be identified in Deep learning, but not in Regression.

We want to keep only meaningful features.

In [25]:
# creating a long list of columns that we want to have
# recreating the dataframe to contain only the mentioned columns
df = df[['Adj. Open', 'Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

# print(df.head())

# defining special relationships, to use as features
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100
df['PCT_Change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100

# creating a new dataframe with the special features
df = df[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume']]

print(df.head())


            Adj. Close    HL_PCT  PCT_Change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.072956    0.324968   44659000.0
2004-08-20   54.322689  7.921706    7.227007   22834300.0
2004-08-23   54.869377  4.049360   -1.227880   18256100.0
2004-08-24   52.597363  7.657099   -5.726357   15247300.0
2004-08-25   53.164113  3.886792    1.183658    9188600.0


Features are kind of like attributes that make up the label.
The label is a prediction

In [31]:
# what is needed to be predicted (The column)
forecast_col = 'Adj. Close'

# fill up empty coloums(columns that don't have numbers)
df.fillna(-99999, inplace=True)

# math.ceil => round a number to the nearest whole number
# try to predict 1% of the dataframe
forecast_out = int(math.ceil(0.01*len(df)))

# will be using data that came 1% days ago to predict today ??

df['label'] = df[forecast_col].shift(-forecast_out)
# shifting the columns negatively (shifting columns up the spreadsheet)
# this way the forecast_col will take an Adj. Close which is 1% days into the future

print(df.head())

# df.dropna() => Drop the rows where at least one element is missing.
# (inplace=True) => Keep the DataFrame with valid entries in the same variable.
df.dropna(inplace=True)
print(df.tail())

            Adj. Close    HL_PCT  PCT_Change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  8.072956    0.324968   44659000.0  67.739104
2004-08-20   54.322689  7.921706    7.227007   22834300.0  69.399229
2004-08-23   54.869377  4.049360   -1.227880   18256100.0  68.752232
2004-08-24   52.597363  7.657099   -5.726357   15247300.0  69.639972
2004-08-25   53.164113  3.886792    1.183658    9188600.0  69.078238
            Adj. Close    HL_PCT  PCT_Change  Adj. Volume   label
Date                                                             
2016-09-22      815.95  1.391017    0.734568    1759290.0  781.10
2016-09-23      814.96  0.523952   -0.022082    1411673.0  802.03
2016-09-26      802.65  1.199776   -0.885382    1472732.0  811.98
2016-09-27      810.73  1.438210    1.109961    1367271.0  805.59
2016-09-28      810.06  0.922117    0.743707    1470280.0  780.29


In ML, can't work with NA(Not available/ no numbers) data.

They need to be replaced with something.
Or get rid of an entire row (not that good in the real world, we'll be loosing data).


---

This is using the previous 1% of days from the entire dataset to predict the Adj. Close of a day (which is 1% of days into the future)

